# Classifying on term frequencies with the Naïve Bayes method

In [30]:
%matplotlib inline

In [1]:
import pandas as pd

df = pd.concat(
    (
        pd.read_csv('../features/trans/tf_trans.csv', index_col=0),
        #pd.read_csv('../features/metadata/tf_keywords.csv', index_col=0),
        #pd.read_csv('../features/metadata/tf_titles.csv', index_col=0),
    ),
    axis=1
)
df = df.groupby(df.columns, axis=1).sum() # Drop duplicate columns
df['tag'] = pd.read_csv('../features/tags.csv', index_col=0)['tag_name']

y = df['tag']
X = df.drop('tag', axis=1)

## Grid search

In [47]:
from sklearn import ensemble
from sklearn import model_selection
from sklearn import naive_bayes

clf = ensemble.BaggingClassifier(
    naive_bayes.MultinomialNB(alpha=0.2),
    n_estimators=10,
    max_samples=0.8,
    max_features=0.8
)

X_train, X_test, y_train, y_test = model_selection.train_test_split(X.values, y.values, test_size=0.2)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

## Precision score

In [48]:
from sklearn import metrics

print(metrics.precision_score(y_test, y_pred, average='micro'))

0.539130434783


## Confusion matrix

In [12]:
import seaborn as sns

cm = metrics.confusion_matrix(y_test, y_pred)
#cm = cm / cm.sum(axis=1).reshape(-1, 1) # Obtain the normalized color map 
ax = sns.heatmap(cm, annot=True, xticklabels=y.unique(), yticklabels=y.unique(), cmap='YlGn')
ax.set_xlabel('Predicted labels', fontweight='bold')
ax.set_ylabel('True labels', fontweight='bold')